In [ ]:
import cv2
import numpy as np

# Initialize video capture for webcam and video source
cap = cv2.VideoCapture(0)
imgTarget = cv2.imread('target3.png')
myVid = cv2.VideoCapture('video.mp4')

detection = False
frameCounter = 0

# Read and resize the video frame to match the target size
success, imgVideo = myVid.read()
hT, wT, _ = imgTarget.shape
imgVideo = cv2.resize(imgVideo, (wT, hT))

# Initialize ORB detector
orb = cv2.ORB_create(nfeatures=1000)
kp1, des1 = orb.detectAndCompute(imgTarget, None)

# Function to stack images
while True:
    success, imgWebcam = cap.read()
    if not success:
        break

    imgAug = imgWebcam.copy()
    kp2, des2 = orb.detectAndCompute(imgWebcam, None)

    if not detection:
        myVid.set(cv2.CAP_PROP_POS_FRAMES, 0)
        frameCounter = 0
    else:
        if frameCounter >= myVid.get(cv2.CAP_PROP_FRAME_COUNT):
            myVid.set(cv2.CAP_PROP_POS_FRAMES, 0)
            frameCounter = 0
        success, imgVideo = myVid.read()
        imgVideo = cv2.resize(imgVideo, (wT, hT))

    if des2 is not None:
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
        good = [m for m, n in matches if m.distance < 0.75 * n.distance]

        if len(good) > 20:
            detection = True
            srcPts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dstPts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
            matrix, mask = cv2.findHomography(srcPts, dstPts, cv2.RANSAC, 5)

            if matrix is not None:
                pts = np.float32([[0, 0], [0, hT], [wT, hT], [wT, 0]]).reshape(-1, 1, 2)
                dst = cv2.perspectiveTransform(pts, matrix)
                imgWebcam = cv2.polylines(imgWebcam, [np.int32(dst)], True, (255, 0, 255), 3)

                imgWarp = cv2.warpPerspective(imgVideo, matrix, (imgWebcam.shape[1], imgWebcam.shape[0]))
                maskNew = np.zeros((imgWebcam.shape[0], imgWebcam.shape[1]), np.uint8)
                cv2.fillPoly(maskNew, [np.int32(dst)], (255, 255, 255))
                maskInv = cv2.bitwise_not(maskNew)
                imgAug = cv2.bitwise_and(imgAug, imgAug, mask=maskInv)
                imgAug = cv2.bitwise_or(imgWarp, imgAug)

    cv2.imshow('Augmented Reality', imgAug)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    frameCounter += 1

cap.release()
myVid.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 